In [ ]:
# %load extract_dataframe.py
import json
import pandas as pd
from textblob import TextBlob
import string
import re

def read_json(json_file: str)->list:
    """
    json file reader to open and read json files into a list
    Args:
    -----
    json_file: str - path of a json file
    
    Returns
    -------
    length of the json file and a list of json
    """
    
    tweets_data = []
    for tweets in open(json_file,'r'):
        tweets_data.append(json.loads(tweets))
    
    
    return len(tweets_data), tweets_data

class TweetDfExtractor:
    """
    this function will parse tweets json into a pandas dataframe
    
    Return
    ------
    dataframe
    """
    def __init__(self, tweets_list):
        
        self.tweets_list = tweets_list

    # an example function
    def find_statuses_count(self)->list:
        statuses_count = []
        for x in self.tweets_list:
            try:
                statuses_count.append(x['user']['statuses_count'])
            except KeyError:
                statuses_count.append(None)
            
        return statuses_count
        
    def find_full_text(self)->list:
        full_text = []
        for x in self.tweets_list:
            try:
                full_text.append(x['full_text'])
            except KeyError:
                full_text.append(None)
            
        return full_text

    def clean_original_text(self, original_text):
        # Make lower case
        clean_text = []
        punctuation = string.punctuation.replace('@', "")
        punctuation = punctuation.replace('_', "")
        
        for x in original_text:
            c = x.lower()
            c = c.replace('\n', '')
            c = c.translate(str.maketrans(" ", " ", punctuation))
            c = re.sub('@[\w]+', ' ', c)
            c = c.translate(str.maketrans(" ", " ", punctuation))
            clean_text.append(c)
        return clean_text
    
    def find_screen_count(self):
        screen_count = []
        for x in self.tweets_list:
            try:
                screen_count.append(x['user']['listed_count'])
            except KeyError:
                screen_count.append(None)
            
        return screen_count
        
    def find_place(self):
        place = []
        for x in self.tweets_list:
            try:
                place.append(x['place']["name"])
            except TypeError:
                place.append(None)
            
        return place

    def find_place_coord_boundaries(self):
        place = []
        for x in self.tweets_list:
            try:
                place.append(x['place']["bounding_box"]["coordinates"])
            except TypeError:
                place.append(None)
            
        return place
    
    def find_place_country(self):
        place = []
        for x in self.tweets_list:
            try:
                place.append(x['place']["country"])
            except TypeError:
                place.append(None)

        return place

    def self_find_place_country(self):
        lang = []
        for x in self.tweets_list:
            try:
                lang.append(x['lang'])
            except KeyError:
                lang.append(None)
            
        return lang

    def find_sentiments(self, text)->list:
        polarity = []
        subjectivity = []
        for x in text:
            t = TextBlob(x)
            polarity.append(t.polarity)
            subjectivity.append(t.subjectivity)
            
        return polarity, subjectivity

    def find_lang(self)->list:
        lang = []
        for x in self.tweets_list:
            try:
                lang.append(x['lang'])
            except KeyError:
                lang.append(None)
            
        return lang
        
    def find_created_time(self)->list:
        created_at = []
        for x in self.tweets_list:
            try:
                created_at.append(x['created_at'])
            except KeyError:
                created_at.append(None)
            
        return created_at

    def find_source(self)->list:
        source = []
        for x in self.tweets_list:
            try:
                s = x['source']
                source.append(s[s.find(">")+1:s.rfind("<")])
            except KeyError:
                source.append(None)
            
        return source

    def find_screen_name(self)->list:
        screen_name = []
        for x in self.tweets_list:
            try:
                screen_name.append(x['user']['screen_name'])
            except KeyError:
                screen_name.append(None)
            
        return screen_name

    def find_followers_count(self)->list:
        followers_count = []
        for x in self.tweets_list:
            try:
                followers_count.append(x['user']['followers_count'])
            except KeyError:
                followers_count.append(None)
            
        return followers_count

    def find_friends_count(self)->list:
        friends_count = []
        for x in self.tweets_list:
            try:
                friends_count.append(x['user']['friends_count'])
            except KeyError:
                friends_count.append(None)
            
        return friends_count

    def is_sensitive(self)->list:
        is_sensitive = []
        for x in self.tweets_list:
            try:
                is_sensitive.append(x['possibly_sensitive'])
            except KeyError:
                is_sensitive.append(None)
            
        return is_sensitive

    def find_favourite_count(self)->list:
        favorite_count = []
        for x in self.tweets_list:
            try:
                favorite_count.append(x['favorite_count'])
            except KeyError:
                favorite_count.append(None)
            
        return favorite_count
    
    def find_retweet_count(self)->list:
        retweet_count = []
        for x in self.tweets_list:
            try:
                retweet_count.append(x['retweet_count'])
            except KeyError:
                retweet_count.append(None)
            
        return retweet_count

    def find_hashtags(self)->list:
        hashtags = []
        for x in self.tweets_list:
            try:
                hashtags.append(" ".join([y["text"] for y in x['entities']['hashtags']]))
            except KeyError:
                hashtags.append(None)
            
        return hashtags

    def find_mentions(self)->list:
        user_mentions = []
        for x in self.tweets_list:
            try:
                user_mentions.append(len(x['entities']['user_mentions']))
            except KeyError:
                user_mentions.append(None)
            
        return user_mentions


    def find_user_location(self)->list:
        location = []
        for x in self.tweets_list:
            try:
                l = x['user']['location']
                if l.strip() == "":
                    location.append(None)
                else:
                    location.append(l)
            except TypeError:
                location.append(None)
            
        return location
        
    def get_tweet_df(self, save=False)->pd.DataFrame:
        """required column to be generated you should be creative and add more features"""
        
        # columns = ['created_at', 'source', 'original_text','polarity','subjectivity', 'lang', 'favorite_count', 'retweet_count',
        #           'original_author', 'followers_count','friends_count','possibly_sensitive', 'hashtags', 'user_mentions', 'place']
        columns = ['created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity',
                   'lang', 'favorite_count', 'retweet_count', 'original_author', 'user_location', 'screen_count',
                   'followers_count','friends_count','possibly_sensitive', 'hashtags', 'user_mentions',
                   'place', 'place_coord_boundaries', 'place_country']
        created_at = self.find_created_time()
        source = self.find_source()
        original_text = self.find_full_text()
        clean_text = self.clean_original_text(original_text)
        polarity, subjectivity = self.find_sentiments(clean_text)
        sentiment = zip(polarity, subjectivity)
        lang = self.find_lang()
        favorite_count = self.find_favourite_count()
        retweet_count = self.find_retweet_count()
        original_author = self.find_screen_name()
        user_location = self.find_user_location()
        screen_count = self.find_screen_count()
        followers_count = self.find_followers_count()
        friends_count = self.find_friends_count()
        possibly_sensitive = self.is_sensitive()
        hashtags = self.find_hashtags()
        user_mentions = self.find_mentions()
        #location = self.find_location()
        place = self.find_place()
        place_coord_boundaries = self.find_place_coord_boundaries()
        place_country = self.find_place_country()

        data = zip(created_at, source, original_text, clean_text, sentiment, polarity, subjectivity, lang,
                   favorite_count, retweet_count, original_author, user_location, screen_count,
                   followers_count, friends_count, possibly_sensitive, hashtags, user_mentions, place
                   , place_coord_boundaries, place_country)
        df = pd.DataFrame(data=data, columns=columns)

        if save:
            df.to_csv('processed_tweet_data.csv', index=False)
            print('File Successfully Saved.!!!')
        
        return df

                
if __name__ == "__main__":
    # required column to be generated you should be creative and add more features
    columns = ['created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang', 'favorite_count', 'retweet_count', 
    'original_author', 'screen_count', 'followers_count','friends_count','possibly_sensitive', 'hashtags', 'user_mentions', 'place', 'place_coord_boundaries']
    _, tweet_list = read_json("data/global_twitter_data.json")
    tweet = TweetDfExtractor(tweet_list)
    tweet_df = tweet.get_tweet_df() 

    # use all defined functions to generate a dataframe with the specified columns above


In [17]:
str.translate

<method 'translate' of 'str' objects>

In [19]:
tweet_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22000 entries, 0 to 21999
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   created_at              22000 non-null  object 
 1   source                  22000 non-null  object 
 2   original_text           22000 non-null  object 
 3   clean_text              22000 non-null  object 
 4   sentiment               22000 non-null  object 
 5   polarity                22000 non-null  float64
 6   subjectivity            22000 non-null  float64
 7   lang                    22000 non-null  object 
 8   favorite_count          22000 non-null  int64  
 9   retweet_count           22000 non-null  int64  
 10  original_author         22000 non-null  object 
 11  user_location           12106 non-null  object 
 12  screen_count            22000 non-null  int64  
 13  followers_count         22000 non-null  int64  
 14  friends_count           22000 non-null

In [18]:
print(str(tweet_df.head()["clean_text"]))
tweet_df.head()["original_text"]

0    rt   extra random image ilets focus in one ver...
1    rt   chinas media explains the military reason...
2    china even cut off communication they dont anw...
3    putin to xijinping  i told you my friend taiwa...
4    rt   i’m sorry i thought taiwan was an indepen...
Name: clean_text, dtype: object


0    RT @i_ameztoy: Extra random image (I):\n\nLets...
1    RT @IndoPac_Info: #China's media explains the ...
2    China even cut off communication, they don't a...
3    Putin to #XiJinping : I told you my friend, Ta...
4    RT @ChinaUncensored: I’m sorry, I thought Taiw...
Name: original_text, dtype: object

In [8]:
# %load clean_tweets_dataframe.py
import pandas as pd

class Clean_Tweets:
    """
    The PEP8 Standard AMAZING!!!
    """
    def __init__(self, df:pd.DataFrame):
        self.df = df
        print('Automation in Action...!!!')
        
    def drop_unwanted_column(self, df:pd.DataFrame)->pd.DataFrame:
        """
        remove rows that has column names. This error originated from
        the data collection stage.  
        """
        unwanted_rows = df[df['retweet_count'] == 'retweet_count' ].index
        df.drop(unwanted_rows , inplace=True)
        df = df[df['polarity'] != 'polarity']
        
        return df
    
    def drop_duplicate(self, df:pd.DataFrame)->pd.DataFrame:
        """
        drop duplicate rows
        """
        df.drop_duplicates()
        
        return df
    
    def convert_to_datetime(self, df:pd.DataFrame)->pd.DataFrame:
        """
        convert column to datetime
        """
        df["created_at"] = pd.to_datetime(df["created_at"], infer_datetime_format=True)
        df = df[df['created_at'] >= '2020-12-31' ]
        
        return df
    
    def convert_to_numbers(self, df:pd.DataFrame)->pd.DataFrame:
        """
        convert columns like polarity, subjectivity, retweet_count
        favorite_count etc to numbers
        """
        
        integer_columns = ["retweet_count", "followers_count", "friends_count", "user_mentions"]
        float_columns = ["polarity", "subjectivity"]
        
        df[integer_columns] = df[integer_columns].astype(int)
        df[float_columns] = df[float_columns].astype(float)
        
        return df
    
    def remove_non_english_tweets(self, df:pd.DataFrame)->pd.DataFrame:
        """
        remove non english tweets from lang
        """
        mask = df["lang"] == "en"
        df = df[mask]
        
        return df
    
    def remove_retweets(self, df:pd.DataFrame)->pd.DataFrame:
        mask = df["original_text"].str.contains("^RT")
        df = df[~mask]
        return df


In [131]:
c = Clean_Tweets(tweet_df)

Automation in Action...!!!


In [133]:
tweet_df["created_at"]
c.remove_retweets(tweet_df).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4736 entries, 2 to 21997
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   created_at              4736 non-null   object 
 1   source                  4736 non-null   object 
 2   original_text           4736 non-null   object 
 3   clean_text              4736 non-null   object 
 4   sentiment               4736 non-null   object 
 5   polarity                4736 non-null   float64
 6   subjectivity            4736 non-null   float64
 7   lang                    4736 non-null   object 
 8   favorite_count          4736 non-null   int64  
 9   retweet_count           4736 non-null   int64  
 10  original_author         4736 non-null   object 
 11  screen_count            4736 non-null   int64  
 12  followers_count         4736 non-null   int64  
 13  friends_count           4736 non-null   int64  
 14  possibly_sensitive      3468 non-null  